In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [2]:
%ls

Listing directory '/'.
      228    boot.py
      223    config.txt
     1913    ledstrippanel.py
     2918    main.py
     2254    stdmqttas.py
      969    utils.py
       15    webrepl_cfg.py


In [ ]:
# Simplify and deploy on dot matrix at front
# check watchdog does or doesn't work
# report mdns not working on esp8266 (and the watchdog)
# check if timeout on wifi allows alternate wifi passwords to be given

# Make sure the firmware is
# %esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

In [ ]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# clk red      D5
# cs  orange   D8
# din yellow   D7
# gnd green    G
# vcc blue     5V

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [2]:
%sendtofile --source wifimqttconnections.txt config.txt
%sendtofile --append config.txt

pinled       -2
pincs        15
pinsck       14
pinmosi      13
numledchars  20

Sent 5 lines (79 bytes) to config.txt.


In [3]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 81 lines (2677 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [4]:
%sendtofile ledstrippanel.py

import framebuf
from machine import Pin, SPI

cs, spi = None, None
brightnesschars = None
buffer = None
fbuff = None
numledchars = 0
# ESP32-HSPI: sck=14, mosi=13

def setbrightness(brightness, i0=0, i1=9999):
    # 0<=brightness<=15
    for i in range(max(i0, 0), min(i1, numledchars)):
        brightnesschars[(numledchars - i)*2 - 1] = brightness
        
def showbrightness():
    cs.value(0)  
    spi.write(brightnesschars)
    cs.value(1)
    
def scrollbrightness(brightness=0):
    for i in range(numledchars - 1):
        brightnesschars[(numledchars - i)*2 - 1] = brightnesschars[(numledchars - (i+1))*2 - 1]
    brightnesschars[1] = brightness
    
s = bytearray(8)
def show():
    for y in range(8):
        s[6] = s[4] = s[2] = s[0] = y+1
        yp = y*(numledchars) + numledchars
        cs.value(0)  # cannot put in loop as resets the shifting
        for m in range(0, numledchars-1, 4):
            s[1] = buffer[yp - m - 1]
            s[3] = buffer[yp - m - 2]
            s[5] = buffer[yp - m - 3]
            s[7] = buffer[yp - m - 4]
            spi.write(s)
        cs.value(1)

def init():
    for i in range(0, 10, 2):
        cs.value(0)
        x = b"\x0c\x00\x0f\x00\x0b\x07\x09\x00\x0c\x01"[i:i+2]
        spi.write(x*numledchars)
        cs.value(1)
    showbrightness()
        
def setup(lnumledchars, lcs, lspi):
    global numledchars, cs, spi, brightnesschars, buffer, fbuff
    cs, spi = lcs, lspi
    numledchars = lnumledchars # can be 4n+1 for scroll buffer

    brightnesschars = bytearray(bytes([0x0a, 0])*numledchars)
    buffer = bytearray(8*(numledchars))  # make one extra character to scroll from
    fbuffwidth = 8*(numledchars)
    fbuffheight = 8
    fbuff = framebuf.FrameBuffer(buffer, fbuffwidth, fbuffheight, framebuf.MONO_HLSB)

    init()
    
    for i in range(0, numledchars*8, 4):
        fbuff.fill_rect(i, (i%8), 4, 4, 1)
    show()
    return fbuff


Sent 64 lines (1913 bytes) to ledstrippanel.py.


In [16]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [17]:
#%sendtofile main.py

from mqtt_as import MQTTClient
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask
import uasyncio as asyncio
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count
from machine import Pin, PWM, SPI
import time
from ledstrippanel import setup, show
#from uasyncio.queues import Queue

numledchars = int(fconfig["numledchars"])
fbuff = setup(numledchars, Pin(int(fconfig["pincs"]), Pin.OUT), SPI(1, 1000000))
flashpinled(5, 300, 300)

topicmessage = topicstem+"/message"
topiccmd = topicstem+"/cmd"
topicstatus = topicstem+"/status"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"

#qmessages = Queue()
qmessages = [ ]
async def messagetask():
    message = " "
    while True:
        if len(qmessages) == 0:
            await asyncio.sleep_ms(100)
            continue
        #prevmessage, message = message, await(qmessages.get())
        prevmessage, message = message, qmessages.pop(0)
        for xchar, (c1, c2) in enumerate(zip(prevmessage, message)):
            if c1 != c2:
                break
        fbuff.fill(0)
        fbuff.text(message, 0, 0, 1)
        for ys in range(7, -1, -1):
            fbuff.fill_rect(xchar*8, 0, (numledchars - xchar)*8, 8, 0)
            fbuff.text(prevmessage[xchar:], xchar*8, ys - 8, 1)
            fbuff.text(message[xchar:], xchar*8, ys, 1)
            show()
            await asyncio.sleep_ms(50)
        await asyncio.sleep_ms(450)

async def cursorflash(client):    
    for i in itertools_count():
        await asyncio.sleep_ms(1000)
        if client.connectioncount and not client.isconnected() and len(qmessages) < 2:
            qmessages.append("wifi fail: %d"%i)
            await asyncio.sleep_ms(5000)
            ipnumber = client._sta_if.ifconfig()[0]
            qmessages.append(ipnumber)
            await asyncio.sleep_ms(4000)
        elif len(qmessages) == 0:
            fbuff.fill_rect((numledchars-1)*8, 7, 8, 1, i%2)
            show()
        if client.isconnected():
            await client.publish(topicstatus, "beat-%d"%i)
        
def callbackcmd(topic, msg, retained):
    print("callbackcmd", topic, msg)
    if topic == topiccmd:
        aloop.create_task(callbackcmdtask(client, topicreply, msg))
    elif len(qmessages) < 5:
        if 1 <= len(msg) <= 50:
            #aloop.create_task(qmessages.put(msg.decode()))
            qmessages.append(msg.decode())

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    qmessages.append("subscribing")
    await client.subscribe(topiccmd)
    await client.subscribe(topicmessage)
    qmessages.append("ip: "+ipnumber)
    await client.publish(topicmessage, "reconnections #%d"%client.connectioncount)
    client.connectioncount += 1
    print("subscribed")
            
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
client = MQTTClient(config)
client.DEBUG = True
client.connectioncount = 0

aloop = asyncio.get_event_loop()
aloop.create_task(messagetask())
aloop.create_task(cursorflash(client))
aloop.create_task(mqttconnecttask(client))
aloop.create_task(flashledconnectedtask(client))
aloop.run_forever()


sleep disable
await connecting to: HUAWEI-B535-2F99 mosquitto.doesliverpool.xyz
Connecting to broker.
Connected to broker.
*** connected:
.Got reliable connection
Connecting to broker.
Connected to broker.
*** connected:
state: 5 -> 0 (0)
rm 0
scandone
.scandone
state: 0 -> 2 (b0)
state: 2 -> 3 (0)
state: 3 -> 5 (10)
add 0
aid 78
cnt 

connected with HUAWEI-B535-2F99, channel 7
dhcp client start...
ip:192.168.8.113,mask:255.255.255.0,gw:192.168.8.1
Checking WiFi integrity.
Checking WiFi integrity.
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 92, in <module>
  File "uasyncio/core.py", line 1, in run_forever
  File "uasyncio/core.py", line 1, in run_until_complete
  File "uasyncio/core.py", line 1, in wait_io_event
KeyboardInterrupt: 


In [18]:
fbuff.fill(0)
show()